In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import requests
import json
import pandas as pd
from pandas import DataFrame, Series


In [15]:
all_seasons = []

for season in range(2021, 2024):
    summary_url = f"https://api.nhle.com/stats/rest/en/skater/summary?limit=-1&cayenneExp=seasonId={season}{season+1}&gameType=2"

    try:
        summary_resp = requests.get(summary_url)
        summary_resp.raise_for_status() 
        summary_json =  summary_resp.json()

        if summary_json['data']:
            df_summary = DataFrame(summary_json['data'])
            all_seasons.append(df_summary)
            df_summary['season'] = f"{season}-{season + 1}"
            print(f"Successfully fetched data for season {season}-{season+1}")
        else:
            print(f"No data returned for season {season}-{season + 1}")
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for season {season}-{season + 1}: {e}")

if all_seasons:
    df_combined_summary = pd.concat(all_seasons, ignore_index=True)
    df_combined_summary = df_combined_summary.groupby('playerId').agg({
            'playerId': 'first',
            'positionCode': 'first',
            'gamesPlayed': 'sum',
            'otGoals': 'sum',
            'gameWinningGoals': 'sum',
        }).reset_index(drop = True)
    
print(df_combined_summary)

Error fetching data for season 2021-2022: Invalid URL 'com': No scheme supplied. Perhaps you meant https://com?
Error fetching data for season 2022-2023: Invalid URL 'com': No scheme supplied. Perhaps you meant https://com?
Error fetching data for season 2023-2024: Invalid URL 'com': No scheme supplied. Perhaps you meant https://com?
      playerId positionCode  gamesPlayed  otGoals  gameWinningGoals
0      8465009            D           72        0                 0
1      8466138            C           35        0                 1
2      8469455            C           76        0                 1
3      8470281            D           80        0                 1
4      8470595            C           93        0                 1
...        ...          ...          ...      ...               ...
1251   8484314            L           20        0                 0
1252   8484321            D            4        0                 0
1253   8484325            C           19        0   

In [16]:
df_combined_summary = df_combined_summary.loc[(df_combined_summary['positionCode'] != 'D') & (df_combined_summary['gamesPlayed'] >= 60)]
df_combined_summary = df_combined_summary.reset_index(drop = True)
df_combined_summary = df_combined_summary.fillna(0)